# **Imports**

In [1]:
import os
os.chdir("../../..")
os.chdir(r"src")

from Polarization.word_partisanship import *
from GloVe.weights import *
from Axes.models import *
from Axes.projection_functions import *
from Axes.axes_definition import *

import numpy as np

import warnings
warnings.filterwarnings("ignore")

os.chdir("../")

# **Polarization log odds ratio**

## **Unigrams**

In [74]:
party_1 = "Lab"
party_2 = "Con" 

In [75]:
dfs_partisanship = []

for i in range(10):
    df = standard_opening(
            "data/FinalDataframes/FilteredFinalDataFrame_201" + str(i) + ".csv", True
        )
    df = df[df["source"] == "par"]
    year = eval("201" + str(i))
    partisanship_matrix, idx2words, words2idx = get_word_partisanship(df, year, party_1, party_2)
    deltas = partisanship_matrix[3, :]
    words = np.array(list(idx2words.values()), dtype=None)
    df_sorted_partisanships = pd.DataFrame({'words' : words, 'deltas' : deltas}).sort_values(by='deltas', ascending=False)
    dfs_partisanship.append(df_sorted_partisanships)

## **Bigrams**

In [2]:
party_1 = "Lab"
party_2 = "Con" 

In [25]:
dfs_partisanship = []

for i in range(10):
    df = standard_opening(
            "data/FinalDataframes/FilteredFinalDataFrame_201" + str(i) + ".csv", True
        )
    df = df[df["source"] == "par"]
    year = eval("201" + str(i))
    partisanship_matrix, idx2words, words2idx = get_word_partisanship(df, year, party_1, party_2, bigram=True)
    deltas = partisanship_matrix[3, :]
    words = np.array(list(idx2words.values()), dtype=None)
    df_sorted_partisanships = pd.DataFrame({'words' : words, 'deltas' : deltas}).sort_values(by='deltas', ascending=False)
    dfs_partisanship.append(df_sorted_partisanships)

# **Combine with embeddings**

In [76]:
def cosine_with_axis_word(word: str, b,  model_words) :   

    array_1 = model_words[word]
    array_2 = b

    cosine = np.dot(array_1, array_2.T) / (norm(array_1) * norm(array_2))

    return cosine

def cosine_with_axis_bigram(bigram: str, b,  model_words) :   

    word_1, word_2 = bigram.split(' ')

    array_1 = (model_words[word_1] + model_words[word_2]) * 0.5
    array_2 = b

    cosine = np.dot(array_1, array_2.T) / (norm(array_1) * norm(array_2))

    return cosine

def get_quantiles(data, percentiles):
    """
    Get quantiles from a distribution.
    
    Parameters:
        data (array-like): The data.
        percentiles (array-like): The percentiles to compute (0-100).
    
    Returns:
        quantiles (array): The values at the specified percentiles.
    """
    return np.percentile(data, percentiles)

def filter_deltas(df, delta_low_percentile, delta_high_percentile, cos_low_percentile, cos_high_percentile):
    delta_percentiles = [delta_low_percentile, delta_high_percentile]
    cos_percentiles = [cos_low_percentile, cos_high_percentile]

    quantiles_deltas = get_quantiles(df['deltas'], delta_percentiles)
    quantiles_cos = get_quantiles(df['cos'], cos_percentiles)

    df = df.loc[(df['deltas'] < quantiles_deltas[0]) | (df['deltas'] > quantiles_deltas[1])]
    df = df.loc[(df['cos'] < quantiles_cos[0]) | (df['cos'] > quantiles_cos[1])]

    return df

## For unigrams

In [77]:
for i in range(10) :

    df = dfs_partisanship[i]
    m = models_w[i]
    pos_a = filter_model(pos_1, m)
    neg_a = filter_model(neg_1, m)

    b = barycentre(pos_a, m) - barycentre(neg_a, m)
    df['cos'] = df['words'].apply(cosine_with_axis_word, b=b, model_words=m)
    dfs_partisanship[i] = df

## For bigrams

In [28]:
for i in range(10) :

    df = dfs_partisanship[i]
    m = models_w[i]
    pos_a = filter_model(pos_1, m)
    neg_a = filter_model(neg_1, m)

    b = barycentre(pos_a, m) - barycentre(neg_a, m)
    df['cos'] = df['words'].apply(cosine_with_axis_bigram, b=b, model_words=m)
    dfs_partisanship[i] = df

## Filtering

In [79]:
df = dfs_partisanship[7]

In [80]:
df.loc[df['words'].str.contains('amazon')]

,words,deltas,cos
10964,amazon,0.007254,0.209501
3526,amazoncom,0.000000,0.141262
13254,amazonian,0.000000,0.166705
12360,amazoncouk,0.000000,0.199117
2607,amazonbest,0.000000,0.178672
12992,amazonamazon,0.000000,0.201914
1828,dealamazon,0.000000,0.385852
7288,amazonfresh,0.000000,0.143035
8432,saidamazon,0.000000,0.070523
11516,amazonth,-0.824044,0.022417


In [65]:
df = filter_deltas(df, 10, 90, 20, 0)

In [66]:
df.sort_values(by='deltas', ascending=False)

,words,deltas,cos
9994,scottish give,0.641854,-0.119336
2376,regim howev,0.641854,-0.189289
9730,within social,0.638231,-0.061350
12791,prefer follow,0.638231,0.008373
13092,subsidiari part,0.638231,0.026633
15921,biggest democrat,0.638231,0.144424
16109,alik deliv,0.638231,0.039942
5873,allow dealer,0.638231,0.029463
12574,announc lastli,0.638231,-0.005192
18950,wrist struck,0.638231,-0.081348


# **Define polarization from embeddings**

## **Polarization as topic embeddings cosine distance**
https://doi.org/10.48550/arXiv.2104.07814